In [2]:
import urllib2
import os
import zipfile
import pandas as pd
import json
import ast

In [ ]:
# download data
def retrieveData():
    url = 'http://crowdcrafting.org/project/landfill/tasks/export?type=task_run&format=csv'
    response = urllib2.urlopen(url)
    output = open('landfill_data.zip', 'wb')
    output.write(response.read())
    output.close()    
    return unzip()

def unzip():
    fh = open('landfill_data.zip', 'rb')
    z = zipfile.ZipFile(fh)
    for name in z.namelist():
        outpath = ""
        z.extract(name, outpath)
    fh.close()
    os.remove('landfill_data.zip')
    
    return load_data(name)

In [3]:
data = pd.read_csv('../landfill_task_run.csv', error_bad_lines=False)

Skipping line 642: expected 20 fields, saw 21



In [150]:
def create_feature(data):
    record = data[0]
    if record != '0':
        record = ast.literal_eval(record)
        
        # loop through records if there are multiple
        for i in record:
            geometry = ast.literal_eval(i['area_geometry'])
            result = {
                "type": "Feature",
                "geometry": geometry,
                "properties": {
                    "area" : i['area'], 
                    "id" : i['id'], 
                    "fac_name" : data[1],
                    "fac_state" : data[2],
                    "fac_street" : data[3],
                    "task_id" : data[4],
                    "task_user_id" : data[5],
                    "task_runinfo__skip" : data[6],
                }
            }
    else:
        result = {
            "type": "Feature",
            "geometry": None,
            "properties": {
                "area" : 0,
                "id" : None,
                "fac_name" : data[1],
                "fac_state" : data[2],
                "fac_street" : data[3],
                "task_id" : data[4],
                "task_user_id" : data[5],
                "task_runinfo__skip" : data[6],
            }
        }
    
    return result

In [151]:
#a stupid hack
temp = zip(
            list(data['task_runinfo__area']),
            list(data['task_runinfo__Facility_Name']),
            list(data['task_runinfo__Facility_State']),
            list(data['task_runinfo__Facility_Street']),
            list(data['task_run__task_id']),
            list(data['task_run__user_id']),
            list(data['task_runinfo__skip']),    
          )

features = list(map(create_feature, temp))
geojson = json.dumps({
    "type": "FeatureCollection",
    "features": features,
})

In [154]:
def writeFile(geojson):
    print "Writing geojson file..."
    outputFile = open('../landfill_geojson.geojson', 'wb')
    outputFile.write(geojson)
    outputFile.close()

In [155]:
writeFile(geojson)

Writing geojson file...
